# Determining brain regions associated with and predictive of moral wrongness

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.io

from nltools.data import Brain_Data
from joblib import Parallel, delayed
from sklearn.metrics import mean_squared_error
from scipy.stats import sem, pearsonr

from nltools.stats import (
                           _calc_pvalue, 
                           correlation_permutation, 
                           summarize_bootstrap, 
                           threshold, 
                           fdr
)

%matplotlib inline

# Univariate Effect

Code for parametric GLMs is documented in `glm_disc_parametric.py`.

In [8]:
parametric_betas = [x for x in glob.glob('/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/glm/ucsb_1/sub*') if 'parametric' in x]
parametric_brain = Brain_Data(parametric_betas)
print(len(parametric_betas))
parametric_brain.ttest(threshold_dict={'fdr':.05})['thr_t'].write("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/activations/parametric_ttest_fdr05.nii.gz")

64


/home/fhopp/.conda/envs/mjs/lib/python3.11/site-packages/nilearn/masking.py:979: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(mask_img, unmasked, affine)


# Multivariate Effect

Code for within-subject item-level GLMs (LSA) documented in `03_glm_trial.py`.

In [49]:
betas_dir = '/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/glm/ucsb_1/'

sub_list = [x.split('/')[-1].split('_')[0] for x in glob.glob(betas_dir+'sub*')]
sub_list = list(set(sub_list))
sub_list.sort()

def svr_within_subject(sub):

    betas = [x for x in glob.glob(betas_dir+'sub*') if 'item' in x and sub in x]
    betas.sort()

    sub_info = pd.DataFrame()
    sub_info['subject'] = pd.Series([int(x.split('/')[-1].split('_')[0].split('-')[1]) for x in betas])
    sub_info['vignette'] = pd.Series([x.split('/')[-1].split('c0')[0].split('vig')[1][1:-1] for x in betas])

    # Load behavior 
    beh_dir = f'/srv/lab/fmri/mofomic/ucsb_1/'
    beh_files = glob.glob(os.path.join(beh_dir, f'{sub}/beh/{sub}_task-mfv_*_beh.tsv'))
    vig_beh_dfs = []
    for file in beh_files:
        df = pd.read_csv(file, sep='\t')
        df['run'] = int(file.split('/')[-1].split('-')[-1].split('_')[0])
        vig_beh_dfs.append(df)
    df_beh = pd.concat(vig_beh_dfs).reset_index(drop=True)
    df_beh = df_beh.dropna()
    df_beh = df_beh[['sub_id','moral_decision','cond_id']]
    df_beh = df_beh.rename(columns={'cond_id':'vignette'})

    sub_info = sub_info.merge(df_beh, on='vignette')
    sub_info = sub_info.rename(columns={'moral_decision':'ratings'})

    sub_data = Brain_Data(betas)
    sub_data.X['subject'] = sub_info['subject']
    sub_data.Y = sub_info['ratings']

    wb_stats = sub_data.standardize(axis=1).predict(algorithm='svr', 
                              cv_dict={'type': 'kfolds','n_folds': 10}, 
                                                  plot=False, **{'kernel':"linear"})

    wb_stats['weight_map'].write(f'/srv/lab/fmri/mft/fhopp_diss/analysis/signature/weightmaps/within_subject/discovery/{sub}.nii.gz')
    with open(f'/srv/lab/fmri/mft/fhopp_diss/analysis/signature/weightmaps/within_subject/discovery/{sub}.pkl', 'wb') as f:
        pickle.dump(wb_stats, f)

    return sub

In [ ]:
# Parallel(n_jobs=-1)(delayed(svr_within_subject)(sub) for sub in sub_list)

overall Root Mean Squared Error: 0.10
overall Correlation: 1.00
overall CV Root Mean Squared Error: 0.91
overall CV Correlation: 0.44
overall Root Mean Squared Error: 0.10
overall Correlation: 0.99
overall CV Root Mean Squared Error: 0.80
overall CV Correlation: 0.24
overall Root Mean Squared Error: 0.09
overall Correlation: 1.00
overall CV Root Mean Squared Error: 0.61
overall CV Correlation: 0.81
overall Root Mean Squared Error: 0.10
overall Correlation: 1.00
overall CV Root Mean Squared Error: 0.72
overall CV Correlation: 0.62
overall Root Mean Squared Error: 0.09
overall Correlation: 1.00
overall CV Root Mean Squared Error: 0.74
overall CV Correlation: 0.66
overall Root Mean Squared Error: 0.10
overall Correlation: 1.00
overall CV Root Mean Squared Error: 0.73
overall CV Correlation: 0.71
overall Root Mean Squared Error: 0.10
overall Correlation: 1.00
overall CV Root Mean Squared Error: 0.91
overall CV Correlation: 0.53
overall Root Mean Squared Error: 0.10
overall Correlation: 1.0

['sub-01',
 'sub-02',
 'sub-03',
 'sub-04',
 'sub-05',
 'sub-06',
 'sub-07',
 'sub-08',
 'sub-09',
 'sub-10',
 'sub-11',
 'sub-12',
 'sub-13',
 'sub-14',
 'sub-15',
 'sub-16',
 'sub-17',
 'sub-18',
 'sub-19',
 'sub-20',
 'sub-21',
 'sub-22',
 'sub-23',
 'sub-24',
 'sub-25',
 'sub-26',
 'sub-27',
 'sub-28',
 'sub-29',
 'sub-30',
 'sub-31',
 'sub-32',
 'sub-33',
 'sub-34',
 'sub-35',
 'sub-36',
 'sub-37',
 'sub-38',
 'sub-39',
 'sub-40',
 'sub-41',
 'sub-42',
 'sub-43',
 'sub-44',
 'sub-45',
 'sub-46',
 'sub-47',
 'sub-48',
 'sub-49',
 'sub-50',
 'sub-51',
 'sub-52',
 'sub-53',
 'sub-54',
 'sub-55',
 'sub-56',
 'sub-57',
 'sub-58',
 'sub-59',
 'sub-60',
 'sub-61',
 'sub-62',
 'sub-63',
 'sub-64']

In [ ]:
mvpa_betas = [x for x in glob.glob('/srv/lab/fmri/mft/fhopp_diss/analysis/signature/weightmaps/within_subject/discovery/*.nii.gz')]
mvpa_betas.sort()

mvpa_brain = Brain_Data(mvpa_betas)
mvpa_brain.ttest(threshold_dict={'fdr':.05})['thr_t'].write("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/activations/within_mvpa_ttest_fdr05.nii.gz")

In [ ]:
# Write out predictions from population model (yhat) to be used in source reconstruction aka model encoding maps
betas_dir = '/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/glm/ucsb_1/'

sub_list = [x.split('/')[-1].split('_')[0] for x in glob.glob(betas_dir+'sub*')]
sub_list.sort()

betas = [x for x in glob.glob(betas_dir+'sub*') if 'common' in x and 'scaled' in x]
betas.sort()

sub_info = pd.DataFrame()
sub_info['subject'] = pd.Series([int(x.split('/')[-1].split('_')[0].split('-')[1]) for x in betas])
sub_info['ratings'] = pd.Series([int(x.split('/')[-1].split('_')[4]) for x in betas])

sub_preds = pd.read_pickle("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/weightmaps/mjs/mjs_weights.pkl")
per_subject_predictions = [x for x in np.array_split(sub_preds['yfit_xval'], len(sub_info.subject.unique()))]

sub_info['yhat'] = pd.Series(sub_preds['yfit_xval'])

for sub in sub_info.subject.unique():
    sub_data = sub_info[sub_info['subject'] == sub]
    sub_preds = sub_data['yhat']
    scipy.io.savemat(f'/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/preds/{sub}.mat', {'yhat': sub_preds.to_numpy().reshape(-1, 1)})

In [ ]:
# Write out predictions from within-subject models (yhat) to be used in source reconstruction
sub_files = glob.glob(("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/weightmaps/within_subject/discovery/sub*.pkl"))
sub_files.sort()

for i, sub_file in enumerate(sub_files):
    sub_data = pd.read_pickle(sub_file)
    sub = sub_file.split('/')[-1].split('.')[0].split('-')[1]
    scipy.io.savemat(f'/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/within_sub_preds/{sub}.mat', {'yhat': sub_data['yfit_xval'].reshape(-1, 1)})

***
# T-test on model encoding maps for population-level MJS

For code used to transform the discovery betas into activation patterns, see `model_encode.m`.

In [3]:
encoding_maps = Brain_Data([x for x in glob.glob("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/weightmaps/model_encode_pop/*")])
encoding_maps.ttest()['t'].write("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/activations/encoding_map_pop.nii.gz")
encoding_maps.ttest(threshold_dict={'fdr':.05})['thr_t'].write("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/activations/encoding_map_pop_fdr05.nii.gz")

/home/fhopp/.conda/envs/mjs/lib/python3.11/site-packages/nilearn/masking.py:979: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(mask_img, unmasked, affine)


# T-test on model encoding maps for within-subject weightmaps

For code used to transform the discovery betas into activation patterns, see `model_encode.m`.

In [4]:
encoding_maps = Brain_Data([x for x in glob.glob("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/weightmaps/model_encode/*")])
encoding_maps.ttest()['t'].write("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/activations/encoding_map_sub.nii.gz")
encoding_maps.ttest(threshold_dict={'fdr':.05})['thr_t'].write("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/activations/encoding_map_sub_fdr05.nii.gz")

/home/fhopp/.conda/envs/mjs/lib/python3.11/site-packages/nilearn/masking.py:979: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(mask_img, unmasked, affine)


# Conjunction (core system) maps 

## Population-Level

In [11]:
mjs = Brain_Data("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/weightmaps/mjs/mjs_bootstrap_thresh_n10000.nii.gz")
mjs_haufe = Brain_Data("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/activations/encoding_map_pop_fdr05.nii.gz")
conj = mjs.copy()
# fill map with 0s
conj.data = np.zeros(mjs.data.shape)

for i, v in enumerate(mjs.data):
    if v > 0 and mjs_haufe.data[i] > 0:
        conj.data[i] = 1
    if v < 0 and mjs_haufe.data[i] < 0:
        conj.data[i] = -1
conj.write("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/activations/mjs_haufe_conj.nii.gz")

## Subject-Level

In [12]:
mjs = Brain_Data("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/activations/within_mvpa_ttest_fdr05.nii.gz")
mjs_haufe = Brain_Data("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/activations/encoding_map_sub_fdr05.nii.gz")
conj = mjs.copy()
# fill map with 0s
conj.data = np.zeros(mjs.data.shape)

for i, v in enumerate(mjs.data):
    if v > 0 and mjs_haufe.data[i] > 0:
        conj.data[i] = 1
    if v < 0 and mjs_haufe.data[i] < 0:
        conj.data[i] = -1
conj.write("/srv/lab/fmri/mft/fhopp_diss/analysis/signature/results/activations/mjs_haufe_sub_conj.nii.gz")